Задание 1: Паттерн Builder (Строитель)

In [1]:
from abc import ABC, abstractmethod

# --- 1. Product ---
class Computer:
    """Комплексный объект, который мы хотим построить."""
    def __init__(self):
        self.cpu = None
        self.ram_gb = None
        self.storage_gb = None
        self.gpu = None
        self.os = None

    def __str__(self):
        return (f"Computer Configuration:\n"
                f"  CPU: {self.cpu or 'Not set'}\n"
                f"  RAM: {self.ram_gb or 'Not set'} GB\n"
                f"  Storage: {self.storage_gb or 'Not set'} GB SSD/HDD\n"
                f"  GPU: {self.gpu or 'Not set'}\n"
                f"  OS: {self.os or 'Not set'}")

# --- 2. Abstract Builder ---
class ComputerBuilder(ABC):
    """Абстрактный строитель, определяющий шаги для создания компьютера."""
    def __init__(self):
        self.computer = Computer()

    @abstractmethod
    def set_cpu(self, cpu_model: str):
        pass

    @abstractmethod
    def set_ram(self, ram_size_gb: int):
        pass

    @abstractmethod
    def set_storage(self, storage_size_gb: int):
        pass

    @abstractmethod
    def set_gpu(self, gpu_model: str):
        pass

    @abstractmethod
    def set_os(self, os_name: str):
        pass

    def get_computer(self) -> Computer:
        return self.computer

# --- 3. Concrete Builder ---
class StandardComputerBuilder(ComputerBuilder):
    """Конкретный строитель для стандартных конфигураций компьютера."""
    def set_cpu(self, cpu_model: str):
        self.computer.cpu = cpu_model
        return self # Для цепочки вызовов (fluent interface)

    def set_ram(self, ram_size_gb: int):
        self.computer.ram_gb = ram_size_gb
        return self

    def set_storage(self, storage_size_gb: int):
        self.computer.storage_gb = storage_size_gb
        return self

    def set_gpu(self, gpu_model: str):
        self.computer.gpu = gpu_model
        return self
    
    def set_os(self, os_name: str):
        self.computer.os = os_name
        return self

# --- 4. Director (Распорядитель) - опциональный, но полезный ---
class ComputerDirector:
    """Управляет процессом сборки, используя строителя."""
    def __init__(self, builder: ComputerBuilder):
        self._builder = builder

    def build_gaming_pc(self):
        self._builder.set_cpu("Intel Core i7") \
                     .set_ram(32) \
                     .set_storage(1000) \
                     .set_gpu("NVIDIA RTX 4070") \
                     .set_os("Windows 11 Pro")
        return self._builder.get_computer()

    def build_office_pc(self):
        self._builder.set_cpu("Intel Core i5") \
                     .set_ram(16) \
                     .set_storage(512) \
                     .set_gpu("Integrated Intel Iris Xe") \
                     .set_os("Windows 11 Home")
        return self._builder.get_computer()

    def build_custom_pc(self, cpu, ram, storage, gpu=None, os="Linux Ubuntu"):
        # Пример использования строителя напрямую для более гибкой конфигурации
        self._builder.set_cpu(cpu)
        self._builder.set_ram(ram)
        self._builder.set_storage(storage)
        if gpu:
            self._builder.set_gpu(gpu)
        self._builder.set_os(os)
        return self._builder.get_computer()

# --- Клиентский код для Задания 1 ---
if __name__ == "__main__":
    print("--- Демонстрация Паттерна Builder ---")

    # Использование Director для создания предопределенных конфигураций
    print("\n1. Создание с помощью Director:")
    gaming_builder = StandardComputerBuilder()
    director = ComputerDirector(gaming_builder)
    
    gaming_pc = director.build_gaming_pc()
    print("\nGaming PC (via Director):")
    print(gaming_pc)

    office_builder = StandardComputerBuilder() # Нужен новый экземпляр строителя или сброс
    director_office = ComputerDirector(office_builder)
    office_pc = director_office.build_office_pc()
    print("\nOffice PC (via Director):")
    print(office_pc)

    # Использование Builder напрямую для кастомной конфигурации
    print("\n2. Создание с помощью Builder напрямую (кастомная сборка):")
    custom_builder = StandardComputerBuilder()
    custom_pc = custom_builder.set_cpu("AMD Ryzen 5") \
                              .set_ram(16) \
                              .set_storage(2000) \
                              .set_os("Arch Linux") \
                              .get_computer() # GPU не указан, будет None
    print("\nCustom PC (direct Builder):")
    print(custom_pc)

    # Использование Director с методом для кастомной сборки
    print("\n3. Создание с помощью Director (кастомная сборка через метод):")
    another_custom_builder = StandardComputerBuilder()
    director_custom = ComputerDirector(another_custom_builder)
    another_custom_pc = director_custom.build_custom_pc(
        cpu="Apple M2", 
        ram=16, 
        storage=512, 
        gpu="Apple M2 Integrated GPU", 
        os="macOS Sonoma"
    )
    print("\nAnother Custom PC (via Director method):")
    print(another_custom_pc)

    print("-" * 50 + "\n")

--- Демонстрация Паттерна Builder ---

1. Создание с помощью Director:

Gaming PC (via Director):
Computer Configuration:
  CPU: Intel Core i7
  RAM: 32 GB
  Storage: 1000 GB SSD/HDD
  GPU: NVIDIA RTX 4070
  OS: Windows 11 Pro

Office PC (via Director):
Computer Configuration:
  CPU: Intel Core i5
  RAM: 16 GB
  Storage: 512 GB SSD/HDD
  GPU: Integrated Intel Iris Xe
  OS: Windows 11 Home

2. Создание с помощью Builder напрямую (кастомная сборка):

Custom PC (direct Builder):
Computer Configuration:
  CPU: AMD Ryzen 5
  RAM: 16 GB
  Storage: 2000 GB SSD/HDD
  GPU: Not set
  OS: Arch Linux

3. Создание с помощью Director (кастомная сборка через метод):

Another Custom PC (via Director method):
Computer Configuration:
  CPU: Apple M2
  RAM: 16 GB
  Storage: 512 GB SSD/HDD
  GPU: Apple M2 Integrated GPU
  OS: macOS Sonoma
--------------------------------------------------



Задание 2: Приложение для приготовления пасты (Порождающие паттерны - используем Builder)

In [2]:
from abc import ABC, abstractmethod
from typing import List

# --- 1. Product: Паста ---
class Pasta:
    """Представляет готовое блюдо - пасту."""
    def __init__(self, name: str):
        self.name = name
        self.pasta_type: str = ""
        self.sauce: str = ""
        self.filling: str | None = None # Начинка может отсутствовать
        self.toppings: List[str] = []

    def __str__(self):
        description = f"--- {self.name} ---\n"
        description += f"Тип пасты: {self.pasta_type}\n"
        description += f"Соус: {self.sauce}\n"
        if self.filling:
            description += f"Начинка: {self.filling}\n"
        if self.toppings:
            description += f"Добавки: {', '.join(self.toppings)}\n"
        else:
            description += "Добавки: нет\n"
        return description

# --- 2. Abstract Builder: PastaBuilder ---
class PastaBuilder(ABC):
    """Абстрактный строитель для пасты."""
    def __init__(self, pasta_name: str):
        self.pasta = Pasta(pasta_name)

    @abstractmethod
    def set_pasta_type(self):
        pass

    @abstractmethod
    def set_sauce(self):
        pass

    @abstractmethod
    def set_filling(self):
        pass

    @abstractmethod
    def add_toppings(self):
        pass

    def get_pasta(self) -> Pasta:
        return self.pasta

# --- 3. Concrete Builders для разных видов пасты ---
class CarbonaraBuilder(PastaBuilder):
    def __init__(self):
        super().__init__("Паста Карбонара")

    def set_pasta_type(self):
        self.pasta.pasta_type = "Спагетти"
        return self

    def set_sauce(self):
        self.pasta.sauce = "Яичный соус с пармезаном и панчеттой/гуанчиале"
        return self

    def set_filling(self):
        self.pasta.filling = None # У Карбонары обычно нет начинки внутри пасты
        return self

    def add_toppings(self):
        self.pasta.toppings.append("Свежемолотый черный перец")
        self.pasta.toppings.append("Дополнительный пармезан")
        return self

class BologneseBuilder(PastaBuilder):
    def __init__(self):
        super().__init__("Паста Болоньезе")

    def set_pasta_type(self):
        self.pasta.pasta_type = "Тальятелле или Феттуччине"
        return self

    def set_sauce(self):
        self.pasta.sauce = "Мясной соус Болоньезе (рагу)"
        return self

    def set_filling(self):
        # Иногда в лазанье болоньезе, но для обычной пасты - нет
        self.pasta.filling = None
        return self

    def add_toppings(self):
        self.pasta.toppings.append("Тертый пармезан")
        self.pasta.toppings.append("Свежий базилик (по желанию)")
        return self

class PestoVeggieBuilder(PastaBuilder):
    def __init__(self):
        super().__init__("Вегетарианская Паста с Песто")

    def set_pasta_type(self):
        self.pasta.pasta_type = "Фузилли или Пенне"
        return self

    def set_sauce(self):
        self.pasta.sauce = "Соус Песто (базилик, кедровые орехи, пармезан, чеснок, оливковое масло)"
        return self

    def set_filling(self):
        self.pasta.filling = "Обжаренные сезонные овощи (например, цуккини, болгарский перец)"
        return self

    def add_toppings(self):
        self.pasta.toppings.append("Кедровые орешки")
        self.pasta.toppings.append("Помидоры черри")
        return self
        
class CustomPastaBuilder(PastaBuilder):
    def __init__(self, name="Моя Кастомная Паста"):
        super().__init__(name)

    def set_pasta_type(self, pasta_type: str):
        self.pasta.pasta_type = pasta_type
        return self

    def set_sauce(self, sauce: str):
        self.pasta.sauce = sauce
        return self

    def set_filling(self, filling: str | None):
        self.pasta.filling = filling
        return self

    def add_toppings(self, toppings: List[str]): # Переопределяем для большей гибкости
        self.pasta.toppings.extend(toppings)
        return self


# --- 4. Director: PastaChef (Повар) ---
class PastaChef:
    """Управляет процессом приготовления пасты."""
    def __init__(self):
        self._builder: PastaBuilder | None = None

    def set_builder(self, builder: PastaBuilder):
        self._builder = builder

    def cook_pasta(self) -> Pasta | None:
        if not self._builder:
            print("Ошибка: Строитель пасты не установлен!")
            return None
        
        # Выполняем шаги в определенном порядке
        self._builder.set_pasta_type()
        self._builder.set_sauce()
        self._builder.set_filling()
        self._builder.add_toppings()
        return self._builder.get_pasta()

# --- Клиентский код для Задания 2 ---
if __name__ == "__main__":
    print("\n--- Демонстрация Приготовления Пасты (Builder) ---")
    chef = PastaChef()

    # Готовим Карбонару
    carbonara_builder = CarbonaraBuilder()
    chef.set_builder(carbonara_builder)
    pasta_carbonara = chef.cook_pasta()
    if pasta_carbonara:
        print(pasta_carbonara)

    # Готовим Болоньезе
    bolognese_builder = BologneseBuilder()
    chef.set_builder(bolognese_builder)
    pasta_bolognese = chef.cook_pasta()
    if pasta_bolognese:
        print(pasta_bolognese)

    # Готовим Вегетарианскую с Песто
    pesto_builder = PestoVeggieBuilder()
    chef.set_builder(pesto_builder)
    pasta_pesto = chef.cook_pasta()
    if pasta_pesto:
        print(pasta_pesto)
        
    # Готовим кастомную пасту напрямую через CustomPastaBuilder
    print("\n--- Кастомная паста через CustomPastaBuilder ---")
    custom_pasta_builder = CustomPastaBuilder("Паста 'Морской Бриз'")
    # У CustomPastaBuilder методы set_... и add_toppings принимают аргументы
    # поэтому Director здесь не так удобен, если он вызывает методы без аргументов.
    # Используем его напрямую.
    pasta_seafood = custom_pasta_builder \
        .set_pasta_type("Лингвини") \
        .set_sauce("Сливочно-чесночный соус") \
        .set_filling("Креветки и мидии") \
        .add_toppings(["Петрушка", "Лимонная цедра"]) \
        .get_pasta()
    print(pasta_seafood)
    
    print("-" * 50 + "\n")


--- Демонстрация Приготовления Пасты (Builder) ---
--- Паста Карбонара ---
Тип пасты: Спагетти
Соус: Яичный соус с пармезаном и панчеттой/гуанчиале
Добавки: Свежемолотый черный перец, Дополнительный пармезан

--- Паста Болоньезе ---
Тип пасты: Тальятелле или Феттуччине
Соус: Мясной соус Болоньезе (рагу)
Добавки: Тертый пармезан, Свежий базилик (по желанию)

--- Вегетарианская Паста с Песто ---
Тип пасты: Фузилли или Пенне
Соус: Соус Песто (базилик, кедровые орехи, пармезан, чеснок, оливковое масло)
Начинка: Обжаренные сезонные овощи (например, цуккини, болгарский перец)
Добавки: Кедровые орешки, Помидоры черри


--- Кастомная паста через CustomPastaBuilder ---
--- Паста 'Морской Бриз' ---
Тип пасты: Лингвини
Соус: Сливочно-чесночный соус
Начинка: Креветки и мидии
Добавки: Петрушка, Лимонная цедра

--------------------------------------------------



Задание 3: Паттерн Prototype (Прототип)

In [3]:
import copy
from abc import ABC, abstractmethod

# --- 1. Abstract Prototype ---
class Prototype(ABC):
    """Абстрактный прототип, объявляющий интерфейс клонирования."""
    @abstractmethod
    def clone(self):
        pass

# --- 2. Concrete Prototype ---
class Document(Prototype):
    """Конкретный прототип - документ."""
    def __init__(self, title: str, content: str, attachments: list, formatting: dict):
        self.title = title
        self.content = content
        self.attachments = attachments  # Список, изменяемый тип
        self.formatting = formatting    # Словарь, изменяемый тип

    def clone(self):
        """Создает клон документа. Важно использовать deepcopy для изменяемых атрибутов."""
        # Простое присваивание или copy.copy() создаст поверхностную копию,
        # где attachments и formatting будут ссылаться на те же объекты.
        return copy.deepcopy(self) # Глубокое копирование

    def __str__(self):
        return (f"Document '{self.title}':\n"
                f"  Content: '{self.content[:30]}...'\n"
                f"  Attachments: {self.attachments}\n"
                f"  Formatting: {self.formatting}")

# --- Клиентский код для Задания 3 ---
if __name__ == "__main__":
    print("\n--- Демонстрация Паттерна Prototype ---")

    # 1. Создаем оригинальный документ (прототип)
    original_formatting = {"font": "Arial", "size": 12, "color": "black"}
    original_attachments = ["image1.png", "datasheet.pdf"]
    
    original_document = Document(
        title="Annual Report 2023",
        content="This is the main content of the annual report for the year 2023...",
        attachments=original_attachments,
        formatting=original_formatting
    )
    print("Оригинальный документ:")
    print(original_document)
    print(f"ID оригинала: {id(original_document)}")
    print(f"ID списка attachments оригинала: {id(original_document.attachments)}")
    print(f"ID словаря formatting оригинала: {id(original_document.formatting)}")

    # 2. Клонируем документ
    cloned_document = original_document.clone()
    print("\nСклонированный документ (сразу после клонирования):")
    print(cloned_document)
    print(f"ID клона: {id(cloned_document)}")
    print(f"ID списка attachments клона: {id(cloned_document.attachments)}") # Должен быть другим
    print(f"ID словаря formatting клона: {id(cloned_document.formatting)}")   # Должен быть другим

    # 3. Модифицируем клон
    cloned_document.title = "Draft Annual Report 2023 (Cloned)"
    cloned_document.content = "This is a DRAFT content..."
    cloned_document.attachments.append("summary.docx") # Изменяем список в клоне
    cloned_document.formatting["color"] = "blue"      # Изменяем словарь в клоне
    cloned_document.formatting["margins"] = {"top": 1, "bottom": 1} # Добавляем в словарь

    print("\nСклонированный документ (после модификаций):")
    print(cloned_document)

    print("\nОригинальный документ (после модификаций клона - должен остаться неизменным):")
    print(original_document) # Проверяем, что оригинал не изменился

    # Демонстрация проблемы с поверхностным копированием (если бы clone использовал copy.copy())
    print("\n--- Демонстрация важности deepcopy ---")
    
    # Создадим метод для поверхностного клонирования для примера
    def shallow_clone(doc: Document):
        return copy.copy(doc)

    shallow_cloned_doc = shallow_clone(original_document)
    print(f"ID attachments оригинала: {id(original_document.attachments)}")
    print(f"ID attachments поверхностного клона: {id(shallow_cloned_doc.attachments)}") # Будут одинаковы!

    # Если изменить attachments в shallow_cloned_doc, изменится и в original_document
    if id(original_document.attachments) == id(shallow_cloned_doc.attachments):
        print("Attachments ссылаются на один объект у оригинала и поверхностного клона.")
        shallow_cloned_doc.attachments.append("VIRUS.exe") # Это изменит и original_document.attachments
        print(f"Attachments оригинала ПОСЛЕ изменения поверхностного клона: {original_document.attachments}")
    else:
        print("Что-то пошло не так с демонстрацией shallow_copy, ID не совпадают.")

    # Восстановим attachments оригинала для чистоты эксперимента
    original_document.attachments = ["image1.png", "datasheet.pdf"] 
    
    print("-" * 50 + "\n")


--- Демонстрация Паттерна Prototype ---
Оригинальный документ:
Document 'Annual Report 2023':
  Content: 'This is the main content of th...'
  Attachments: ['image1.png', 'datasheet.pdf']
  Formatting: {'font': 'Arial', 'size': 12, 'color': 'black'}
ID оригинала: 130742678050752
ID списка attachments оригинала: 130742678554368
ID словаря formatting оригинала: 130742678188032

Склонированный документ (сразу после клонирования):
Document 'Annual Report 2023':
  Content: 'This is the main content of th...'
  Attachments: ['image1.png', 'datasheet.pdf']
  Formatting: {'font': 'Arial', 'size': 12, 'color': 'black'}
ID клона: 130742678054976
ID списка attachments клона: 130742678191232
ID словаря formatting клона: 130742678175232

Склонированный документ (после модификаций):
Document 'Draft Annual Report 2023 (Cloned)':
  Content: 'This is a DRAFT content......'
  Attachments: ['image1.png', 'datasheet.pdf', 'summary.docx']
  Formatting: {'font': 'Arial', 'size': 12, 'color': 'blue', 'margi